In [2]:
from roboticstoolbox import *
from spatialmath import *
from math import pi
import numpy as np
import matplotlib.pyplot as plt
%matplotlib osx

In [3]:
DFbot = DHRobot(
    [
        RevoluteDH(d=0.04145,alpha=np.pi/2,qlim=np.array([-np.pi,np.pi])),
        RevoluteDH(a=-0.08285,qlim=np.array([-np.pi,np.pi])),
        RevoluteDH(a=-0.08285,qlim=np.array([-np.pi,np.pi])),
        RevoluteDH(alpha=-np.pi/2,qlim=np.array([-np.pi,np.pi])),
        RevoluteDH(d=0.11,qlim=np.array([-np.pi,3/2*np.pi]))
    ],
    name = "DFbot",
)
print(DFbot)

DHRobot: DFbot, 5 joints (RRRRR), dynamics, standard DH parameters
┌─────┬─────────┬──────────┬────────┬─────────┬────────┐
│ θⱼ  │   dⱼ    │    aⱼ    │   ⍺ⱼ   │   q⁻    │   q⁺   │
├─────┼─────────┼──────────┼────────┼─────────┼────────┤
│  q1 │ 0.04145 │        0 │  90.0° │ -180.0° │ 180.0° │
│  q2 │       0 │ -0.08285 │   0.0° │ -180.0° │ 180.0° │
│  q3 │       0 │ -0.08285 │   0.0° │ -180.0° │ 180.0° │
│  q4 │       0 │        0 │ -90.0° │ -180.0° │ 180.0° │
│  q5 │    0.11 │        0 │   0.0° │ -180.0° │ 270.0° │
└─────┴─────────┴──────────┴────────┴─────────┴────────┘

┌──┬──┐
└──┴──┘



In [7]:
state0=[0,0.02,0,0,0]
T0=DFbot.fkine(state0)
state1=[0.0000001,0.02,0,0,0]
T1=DFbot.fkine(state1)

In [9]:
print(T0,"\n\n",T1)

   0.9998    0        -0.02     -0.1679    
   0         1         0         0         
   0.02      0         0.9998    0.1481    
   0         0         0         1         
 

    0.9998   -1e-07    -0.02     -0.1679    
   9.998e-08  1        -2e-09    -1.679e-08  
   0.02      0         0.9998    0.1481    
   0         0         0         1         



In [14]:
drq=np.array((T1-T0)/0.0000001)
print(np.round(drq,5))

[[-0.      -1.       0.       0.     ]
 [ 0.9998  -0.      -0.02    -0.16787]
 [ 0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]]


In [21]:
vx,vy,vz=drq[0:3,-1]
print(drq[0:3,-1].round(5))

[ 0.      -0.16787  0.     ]


In [23]:
drq[0:3,0:3].round(5)

array([[-0.    , -1.    ,  0.    ],
       [ 0.9998, -0.    , -0.02  ],
       [ 0.    ,  0.    ,  0.    ]])

In [26]:
R = np.array(T0)[0:3,0:3]
R.round(5)

array([[ 0.9998, -0.    , -0.02  ],
       [ 0.    ,  1.    , -0.    ],
       [ 0.02  , -0.    ,  0.9998]])

In [32]:
#Scew-symmetric matrix: S(w)=drq*inv(R)
S=drq[0:3,0:3].dot(np.linalg.inv(R))
S

array([[-4.99701659e-08, -1.00000000e+00,  6.80428475e-12],
       [ 1.00000000e+00, -4.99600361e-08, -1.89346430e-18],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [42]:
#Find angular velocity wx,wy,wz with respect to base coordinate system.
from spatialmath.base import vex
print("wx,wy,wz:",vex(S).round(5))
wx,wy,wz=vex(S)[0:3]

wx,wy,wz: [0. 0. 1.]


In [49]:
#Jacobian matrix with respect to base coordinate
#Each column:vx,vy,vz,wx,wy,wz
print("vx,vy,vz,wx,wy,wz:",vx.round(5),vy.round(5),vz.round(5),wx.round(5),wy.round(5),wz.round(5))
J0=DFbot.jacob0(state0)
J0.round(5)

vx,vy,vz,wx,wy,wz: 0.0 -0.16787 0.0 0.0 0.0 1.0


array([[ 0.     , -0.10666, -0.10832, -0.10998,  0.     ],
       [-0.16787, -0.     , -0.     , -0.     ,  0.     ],
       [ 0.     , -0.16787, -0.08503, -0.0022 ,  0.     ],
       [ 0.     , -0.     , -0.     , -0.     , -0.02   ],
       [-0.     , -1.     , -1.     , -1.     , -0.     ],
       [ 1.     ,  0.     ,  0.     ,  0.     ,  0.9998 ]])

In [51]:
#End jacobian: with respect to the end effector coordinate
Je = DFbot.jacobe(state0)
Je.round(5)

array([[-0.     , -0.11   , -0.11   , -0.11   ,  0.     ],
       [-0.16787, -0.     , -0.     ,  0.     ,  0.     ],
       [-0.     , -0.1657 , -0.08285,  0.     ,  0.     ],
       [ 0.02   ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-0.     , -1.     , -1.     , -1.     ,  0.     ],
       [ 0.9998 ,  0.     ,  0.     ,  0.     ,  1.     ]])

In [57]:
zero = np.zeros((3,3))
TT = np.zeros((6,6))
TT[0:3,0:3]=R
TT[0:3,3:6]=zero
TT[3:6,0:3]=zero
TT[3:6,3:6]=R
TT.round(5)

array([[ 0.9998, -0.    , -0.02  ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    , -0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.02  , -0.    ,  0.9998,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.9998, -0.    , -0.02  ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  1.    , -0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.02  , -0.    ,  0.9998]])

In [61]:
print("J0:\n",J0.round(5))
print("TT*Je:\n",(TT@Je).round(5))
print("J0-TT*Je=\n",(J0-TT@Je).round(2))

J0:
 [[ 0.      -0.10666 -0.10832 -0.10998  0.     ]
 [-0.16787 -0.      -0.      -0.       0.     ]
 [ 0.      -0.16787 -0.08503 -0.0022   0.     ]
 [ 0.      -0.      -0.      -0.      -0.02   ]
 [-0.      -1.      -1.      -1.      -0.     ]
 [ 1.       0.       0.       0.       0.9998 ]]
TT*Je:
 [[ 0.      -0.10666 -0.10832 -0.10998  0.     ]
 [-0.16787 -0.      -0.      -0.       0.     ]
 [ 0.      -0.16787 -0.08503 -0.0022   0.     ]
 [ 0.      -0.      -0.      -0.      -0.02   ]
 [-0.      -1.      -1.      -1.      -0.     ]
 [ 1.       0.       0.       0.       0.9998 ]]
J0-TT*Je=
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
